In [24]:
import requests
import pandas as pd
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy
import time

In [25]:
def get_conn_postgres(SSH_required,key_path):   #for getting a connection as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [26]:
def get_conn_readreplica(SSH_required,key_path):   #for getting a connection as a result

    db='datawarehouse'
    DB_HOST='gs-prod-new-db-read.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [27]:
def get_df_from_sql_postgres(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [28]:
def get_df_from_sql_readreplica(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='gs-prod-new-db-read.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [29]:
def make_request(address, city):
    url = 'http://grow-simplee-nlb-prod-a264c46571856f67.elb.ap-south-1.amazonaws.com:9016/parse_address'
    headers = {'Content-Type': 'application/json'}
    data = {
        "address": address,
        "city": city
    }
    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # Raise an error for bad status codes
        
        response_json = response.json()
        
        # Extract lat_lng from the response JSON
        lat_lng = response_json.get('geo_entities', {}).get('lat_lng', None)
        return lat_lng if lat_lng else None
        
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        print(f"Response content: {response.content}")
        return None

In [30]:
# Usage with the actual path to the private key
SSH_required = 'Yes'
key_path = '/home/rajat/Downloads/tunnel-ssh .cer'
query = "select * from public.warehouses_warehouse"

# Establish a connection
conn = get_conn_readreplica(SSH_required, key_path)

# Retrieve data into a DataFrame
df_warehouse = get_df_from_sql_readreplica(SSH_required, query, key_path)
# df_warehouse = df_warehouse.iloc[:2]

# Now you can perform further operations with the DataFrame 'df_warehouse'
print(df_warehouse.head())

Tunnel Started
Connection Made
   warehouse_int_id                  warehouse_id  \
0              1109                  Aditya Gupta   
1              1461    Harihar Bhiwandi Warehouse   
2              1471  S0002 Bagru Hastkala Printer   
3              1467                        Source   
4                31           RHJ(Jagraon_Punjab)   

                 warehouse_name  warehouse_gstin  warehouse_phone  \
0                  Aditya Gupta  27AACCW4507F1ZG       9867033153   
1    Harihar Bhiwandi Warehouse  NOGST12345ABCDE       9022747957   
2  S0002 Bagru Hastkala Printer  08ATPPC2484A1ZV       9929619944   
3                        Source  763872749872983  728197387298133   
4         RHJ (Jagraon, Punjab)                        8768746120   

                                   warehouse_address warehouse_city  \
0  601 Abhijeet building, 1, Vaikunthlal Mehta Rd...         Mumbai   
1  Sanghvi Beauty and Technologies Pvt Ltd, Harih...          Thane   
2  Bagru Hastkala Prin

In [31]:
query = "select * from application_db.node"
# Establish a connection
conn = get_conn_postgres(SSH_required, key_path)

# Retrieve data into a DataFrame
df_hubs = get_df_from_sql_postgres(SSH_required, query, key_path)
# df_hubs = df_hubs.iloc[:2]

# Now you can perform further operations with the DataFrame 'df_warehouse'
print(df_hubs.head())

Tunnel Started
Connection Made
   node_id  warehouse_id       node_name  location_id    team_names  \
0       19           359     DS HYD SRNG            1  HYD_BLZ_SRNG   
1       37           359  BRBN-Franchise            1  HYD_FRH_BRBN   
2       48           359  BACP-Franchise            1  HYD_FRH_BACP   
3       49           182     DS PNQ KLWD            1  PNQ_BLZ_KLWD   
4       30          1373  ONFC-Franchise            1  DEL_FRH_ONFC   

   fuel_rate  distance_limit sort_codes  \
0          3             100   HYD/SRNG   
1          0               0   HYD/BRBN   
2          0               0   HYD/BACP   
3          3             100   PNQ/KLWD   
4          0               0   DEL/ONFC   

                                             address        contact_name  \
0  7-1-396/2/1 & 2/H3 Bk Guda Sanjeeva Reddy Naga...           Siddharth   
1  NO.109,P-TYPE,SITE -1 COLONY ,ARUNA ENCLAVE BO...           Siddharth   
2  3, 5-31, near MASJID, MEDCHAL, Rajiv Gandhi Na...   

In [ ]:
warehouse_lat_long_list = []

for index, row in df_warehouse.iterrows():
    lat_long = make_request(row['warehouse_address'], row['warehouse_city'])
    warehouse_lat_long_list.append(lat_long)
    time.sleep(5)  # Delay between requests to avoid overloading the server

# Add the results to the DataFrame
df_warehouse['lat_long'] = warehouse_lat_long_list

# Print the DataFrame with latitude and longitude
print("\nDataFrame with lat_long:")
print(df_warehouse)

In [ ]:
hubs_lat_long_list = []

for index, row in df_hubs.iterrows():
    lat_long = make_request(row['address'], row['city_name'])
    hubs_lat_long_list.append(lat_long)
    time.sleep(5)  # Delay between requests to avoid overloading the server

# Add the results to the DataFrame
df_hubs['lat_long'] = hubs_lat_long_list

# Print the DataFrame with latitude and longitude
print("\nDataFrame with lat_long:")
print(df_hubs)

In [ ]:
# Save DataFrame to a CSV file
csv_file_path = 'df_warehouse.csv'
df_warehouse.to_csv(csv_file_path, index=False)

# Provide a link to download the CSV file
print(f"Warehouse file saved as {csv_file_path}.")

In [ ]:
# Save DataFrame to a CSV file
csv_file_path = 'df_hubs.csv'
df_hubs.to_csv(csv_file_path, index=False)

# Provide a link to download the CSV file
print(f"Hubs file saved as {csv_file_path}.")